In [1]:
# %cd /mnt/data/anhvth5/projects/eedi/

/mnt/data/anhvth5/projects/eedi


In [ ]:
import argparse
import random
from dataclasses import dataclass

import dspy
from eedi.dspy import prog
import llm_utils
from dotenv import load_dotenv
from llm_utils import *
from speedy_utils.all import *

from eedi import dspy as edspy
from eedi.common import *
from eedi.dspy.generate_train_data import (SignatureFillReasoningAndPlan,
                                           SignatureProblemGenerator)
from eedi.models.openai_embed import get_miscon_db_openai

load_dotenv()
setup_logger("E")
lm72 = edspy.LocalLM("QW72B")
dspy.settings.configure(lm=lm72)
def generate_problem_with_demos(miscon, args, plan_reason_df):
    examples = []
    # try:
    # Collect examples based on misconceptions
    for similar_misconception, score in db(miscon):
        if args.skip_exact_match and similar_misconception == miscon:
            continue
        ids = plan_reason_df["MISCONCEPTION_NAME"] == similar_misconception
        if ids.sum() > 0:
            examples.append(plan_reason_df[ids].sample(1, random_state=args.seed).iloc[0].to_dict())
    
    # Generate problem with collected examples
    pred = prog(MISCONCEPTION_NAME=miscon, demos=examples)
    pred = pred.toDict()
    pred["MISCONCEPTION_NAME"] = miscon
    return pred


def get_example(row_parsed):
    # Define the keys that should be extracted from each row
    keep_keys = ["MISCONCEPTION_NAME", "QUESTION", "CORRECT_ANSWER", "STUDENT_WRONG_ANSWER"]
    
    # Create a dictionary from the row_parsed based on the specified keys
    data = {key: row_parsed.get(key, "MISSING_VALUE") for key in keep_keys}
    
    # Return an instance of Example with the extracted data and default values for missing fields
    return dspy.Example(
        base=SignatureFillReasoningAndPlan, 
        **data,
        reasoning="MISSING_VALUE", 
        PLANING="MISSING_VALUE"
    )



2024-11-21 19:44:10.323 | INFO     | eedi.common:<module>:429 - common: ['VectorDBRetriever', 'prepare_db_train_val', 'preproc_df', 'get_df_parsed', 'format_df_generic', 'compute_metric', 'mine_hard_negatives', 'mine_hard_negatives', 'extract_tag', 'format_input_v2', 'format_input_v3', 'TEMPLATE_INPUT_V3', 'construct_base_prompt_v3', 'get_synthetic_df', 'format_docs', 'get_parsed_df_v3', 'df_mapping', 'get_parsed_df_v4', 'get_openai_misconception_db', 'update_train_df', 'load_latest_df', 'OpenAIModel', 'LocalLM']


In [3]:
df_parsed = get_parsed_df_v4()
df_parsed.head(1)


,PROMPT,QUESTION,CORRECT_ANSWER,STUDENT_WRONG_ANSWER,MISCONCEPTION_NAME,MISCONCEPTION_ID,is_val,SubjectName,ConstructName,SubjectNameNew
id,,,,,,,,,,
0_B,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),None,NaN,False,BIDMAS,Use the order of operations to carry out calcu...,Number Basics & Operations


In [4]:
df_parsed

,PROMPT,QUESTION,CORRECT_ANSWER,STUDENT_WRONG_ANSWER,MISCONCEPTION_NAME,MISCONCEPTION_ID,is_val,SubjectName,ConstructName,SubjectNameNew
id,,,,,,,,,,
0_B,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),None,NaN,False,BIDMAS,Use the order of operations to carry out calcu...,Number Basics & Operations
0_C,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times(2+4-5) \),None,NaN,False,BIDMAS,Use the order of operations to carry out calcu...,Number Basics & Operations
0_D,Question: \[\n3 \times 2+4-5\n\]\nWhere do the...,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),Does not need brackets,"Confuses the order of operations, believes add...",1672.0,False,BIDMAS,Use the order of operations to carry out calcu...,Number Basics & Operations
1_A,"Question: Simplify the following, if possible:...","Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+1 \),Does not know that to factorise a quadratic ex...,2142.0,False,Simplifying Algebraic Fractions,Simplify an algebraic fraction by factorising ...,Advanced Algebra
1_B,"Question: Simplify the following, if possible:...","Simplify the following, if possible: \( \frac{...",Does not simplify,\( m+2 \),Thinks that when you cancel identical terms fr...,143.0,False,Simplifying Algebraic Fractions,Simplify an algebraic fraction by factorising ...,Advanced Algebra
...,...,...,...,...,...,...,...,...,...,...
1867_C,Question: Tom and Katie are discussing congrue...,Tom and Katie are discussing congruence and si...,Only Katie,Both Tom and Katie,Thinks congruent shapes are the same shape but...,2312.0,False,Congruency in Other Shapes,Distinguish between congruency and similarity,Geometry & Measures
1867_D,Question: Tom and Katie are discussing congrue...,Tom and Katie are discussing congruence and si...,Only Katie,Neither is correct,Thinks congruent shapes are the same shape but...,2312.0,False,Congruency in Other Shapes,Distinguish between congruency and similarity,Geometry & Measures
1868_A,Question: Jo and Paul are arguing about how to...,Jo and Paul are arguing about how to fully des...,Only Paul,Only\nJo,Forgets to give the centre of rotation,801.0,False,Rotation,Describe a 90° or 270° rotation giving the ang...,Geometry & Measures


## Prepare initial program with reaosning/planning (missing)
Ask chatgpt to fill up with the prompt `at the top`

In [ ]:


# List to store generated examples
examples = []

# Iterate over a sample of 10 rows from df_parsed (dropping rows with NaN values)
for i, row_parsed in df_parsed.dropna().sample(10, random_state=42).iterrows():
    example = get_example(row_parsed)
    examples.append(example)


row_parsed

prog = dspy.ChainOfThought(SignatureFillReasoningAndPlan)
prog._predict.demos = examples
prog.save('./dspy_program/generate_example_missing')
print('***Now copy the content of the file then ask chatgpt***')

### System Prompt: Filling Missing Values in dspy Program

You are an expert in mathematical misconceptions and prompt engineering. Your task is to analyze and fill in missing fields in the provided `dspy` program. The goal is to ensure that all outputs align with the objective of diagnosing common student misconceptions through plausible math problems. Your approach should include clearly defined steps for reasoning and planning, emphasizing consistency, accuracy, and pedagogical clarity.

### Instructions:

1. **Understand the Context and Goal:** 
   - The primary goal of the `dspy` program is to create math problems that reveal student misunderstandings. The problems should be designed to provide both a correct solution and a plausible incorrect solution based on a given misconception.

2. **Analyze the Provided Fields:** 
   - Review the inputs (`MISCONCEPTION_NAME`) and provided problem details (`QUESTION`, `CORRECT_ANSWER`, `STUDENT_WRONG_ANSWER`). Ensure you comprehend the nature of the misconception and the mathematical concept it relates to.

3. **Generate Reasoning:**
   - **Purpose:** Explain why the specific problem targets this misconception. Connect the misconception to the student's likely wrong answer and describe why a student might fall into this error.
   - **Structure:** 
     - Start with the nature of the misconception (e.g., misunderstanding coefficients in brackets).
     - Clarify the student's thought process that leads to the incorrect answer.
     - Reinforce the connection between the incorrect answer and the misconception.

   - **Example Template for Reasoning:**  
     "The student may incorrectly [describe student’s action/thought process]. This results in [wrong answer] due to [specific misconception]."

4. **Develop Planning (PLANING) Steps:**
   - **Purpose:** Outline a clear, step-by-step plan used to design the problem. Focus on leveraging the misconception to create a problem that directly assesses the student's understanding.
   - **Structure:**
     - Identify the specific misconception.
     - Establish the core concept or principle related to this misconception.
     - Describe how the problem is constructed to target the misconception.
     - Confirm how the correct and incorrect answers align with the identified misconception.

   - **Example Template for Planning:**  
     "1. Identify the misconception that [state the misconception].  
      2. Determine the related concept [describe the concept].  
      3. Create a problem involving [details of the question type/structure].  
      4. Establish the correct answer based on [briefly describe the correct method].  
      5. Generate a plausible wrong answer reflecting [explain how it represents the misconception]."

5. **Maintain Consistency and Pedagogical Rigor:**
   - Ensure that each entry consistently relates the misconception to the incorrect answer in a meaningful way.
   - Keep explanations concise, but detailed enough to convey the essence of the misconception and the logic behind the correct and incorrect answers.
   - Validate that the generated incorrect answer is plausible and aligns with the type of error a student would realistically make based on the given misconception.

6. **Output Formatting:**
   - Return the completed fields with clear, well-structured text.
   - Ensure that technical notation or imagery is described accurately for clarity.
  
By following this approach, you’ll generate coherent and effective `reasoning` and `PLANING` fields that align with the provided data, ensuring a robust diagnostic assessment of student misconceptions.
```
---
{
  "self": {
    "lm": null,
    "traces": [],
    "train": [],
    "demos": [
      {
        "MISCONCEPTION_NAME": "Believes that fractions with larger denominators are greater in size",
        "QUESTION": "What sign would make this statement true?\n\\(\n\\frac{4}{6} ? \\frac{10}{15}\n\\)",
        "CORRECT_ANSWER": "\\( = \\)",
        "STUDENT_WRONG_ANSWER": "\\( < \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Does not recognise that choosing 2 items at once means the first item does not get replaced in a probability question",
        "QUESTION": "There are \\( 4 \\) red balls and \\( 3 \\) yellow balls in a cup ![A cup with 4 red and 3 yellow balls in it. All of the balls are the same size.]() Kate picks out three balls at the same time.\n\nShe wants to work out the probability all the balls are red.\n\nWhat is the denominator of the final unsimplified answer?",
        "CORRECT_ANSWER": "\\( 210 \\)",
        "STUDENT_WRONG_ANSWER": "\\( 343 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Beileves you can use the full length when finding an area of part of a composite shape, when part of the length is needed",
        "QUESTION": "What is the area of the shaded section of this composite shape made from rectangles? ![A composite shape that makes a backwards \"L\" shape from 2 rectangles.  The base of the shape is horizontal and is 11cm long. The vertical height of the whole shape is 12cm. The horizontal width of the top part of the shape is 5cm. The vertical height of the bottom rectangle is labelled 4cm. The top rectangle is shaded.]()",
        "CORRECT_ANSWER": "\\( 40 \\mathrm{~cm}^{2} \\)",
        "STUDENT_WRONG_ANSWER": "\\( 60 \\mathrm{~cm}^{2} \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Does not recognise when to find a multiple from a worded question",
        "QUESTION": "There are \\( 7 \\) shelves of plates. Three of the shelves hold \\( 17 \\) plates each. Two of the shelves hold \\( 21 \\) plates each. The other shelves hold \\( 19 \\) plates each.\nHow many plates are there in total?",
        "CORRECT_ANSWER": "\\( 131 \\)",
        "STUDENT_WRONG_ANSWER": "\\( 57 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "When solving an equation, uses the same operation rather than the inverse.",
        "QUESTION": "Solve the equation:\n\\(\n10=\\frac{5}{p}\n\\)",
        "CORRECT_ANSWER": "\\( p=0.5 \\)",
        "STUDENT_WRONG_ANSWER": "\\( p=50 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Does not apply the inverse function to every term in an expression",
        "QUESTION": "![A function machine with 4 rectangles in a row, joined by arrows pointing from left to right. The first rectangle on the left is empty and says \"input\" above it. The next rectangle has \"\u2212 4\" written inside it, the next rectangle has \"\u2715 4\" written inside it and the final rectangle on the right has \"output\" written above it and \"8m \u2212 20\" written inside it.]() What is the input of this function machine?",
        "CORRECT_ANSWER": "\\( 2 m-1 \\)",
        "STUDENT_WRONG_ANSWER": "\\( 2 m-20 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Has written the value of the 'ones' column as the answer, withiut using the numbers in the question.",
        "QUESTION": "\\( \\begin{array}{r}\\quad58 \\\\ \\times 22 \\\\ \\hline\\end{array} \\) What number will be in the ones\/units column of the final answer?",
        "CORRECT_ANSWER": "\\( 6 \\)",
        "STUDENT_WRONG_ANSWER": "\\( 1 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "When solving an equation, uses the same operation rather than the inverse.",
        "QUESTION": "Rearrange the following equation to make \\( v \\) the subject\n\\(F=\\frac{v^{2}}{r}\\)",
        "CORRECT_ANSWER": "\\( v=\\sqrt{r F} \\)",
        "STUDENT_WRONG_ANSWER": "\\( v=r F^{2} \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Believes subtraction is commutative ",
        "QUESTION": "Miran says the following:\n\n\u201cIt doesn't matter if you do \\( 81-53 \\) or \\( 53-81 \\), you get the same answer.\"\n\nWhat do you think?",
        "CORRECT_ANSWER": "Miran is incorrect because subtraction is not commutative",
        "STUDENT_WRONG_ANSWER": "Miran is correct because subtraction is commutative",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Ignores numbers written as words in worded questions, and just calculates using those written in digits",
        "QUESTION": "There are \\( 8 \\) shelves of plates. Four of the shelves hold \\( 15 \\) plates each. Two of the shelves hold \\( 23 \\) plates each. The other shelves hold \\( 17 \\) plates each.\nHow many plates are there in total?",
        "CORRECT_ANSWER": "\\( 140 \\)",
        "STUDENT_WRONG_ANSWER": "\\( 63 \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      }
    ],
    "signature_instructions": "Generate a plausible math problem based on a given misconception.\n\nHere is some meta data for the construct name and subject name for you\n<META_DATA>\n<CONSTRUCT_NAME>\nExamples of construct names:\n    - Use the order of operations to carry out calculations involving powers.\n    - Simplify an algebraic fraction by factorising the numerator.\n    - Calculate the range from a list of data.\n    - Recall and use the intersecting diagonals properties of a rectangle.\n    - Substitute positive integer values into formulae involving powers or roots.\n    - ...\n<\/CONSTRUCT_NAME>\n<SUBJECT_NAME>\n    - BIDMAS (Order of Operations)\n    - Simplifying Algebraic Fractions\n    - Range and Interquartile Range from a List of Data\n    - Properties of Quadrilaterals\n    - Substitution into Formula\n    - Area of Simple Shapes\n    - Converting between Fractions and Percentages\n    - Multiplying and Dividing with Decimals\n    - ...\n<\/SUBJECT_NAME>\n<\/META_DATA>\n\nNOTE: Be *creative* when generate the questions. Just make sure it align with the *given misconception*",
    "signature_prefix": "PLANING:",
    "extended_signature_instructions": "Generate a plausible math problem based on a given misconception.\n\nHere is some meta data for the construct name and subject name for you\n<META_DATA>\n<CONSTRUCT_NAME>\nExamples of construct names:\n    - Use the order of operations to carry out calculations involving powers.\n    - Simplify an algebraic fraction by factorising the numerator.\n    - Calculate the range from a list of data.\n    - Recall and use the intersecting diagonals properties of a rectangle.\n    - Substitute positive integer values into formulae involving powers or roots.\n    - ...\n<\/CONSTRUCT_NAME>\n<SUBJECT_NAME>\n    - BIDMAS (Order of Operations)\n    - Simplifying Algebraic Fractions\n    - Range and Interquartile Range from a List of Data\n    - Properties of Quadrilaterals\n    - Substitution into Formula\n    - Area of Simple Shapes\n    - Converting between Fractions and Percentages\n    - Multiplying and Dividing with Decimals\n    - ...\n<\/SUBJECT_NAME>\n<\/META_DATA>\n\nNOTE: Be *creative* when generate the questions. Just make sure it align with the *given misconception*",
    "extended_signature_prefix": "PLANING:"
  }
}


In [17]:
!cat './dspy_program/generate_example_missing'

{
  "self": {
    "lm": null,
    "traces": [],
    "train": [],
    "demos": [
      {
        "MISCONCEPTION_NAME": "Believes that fractions with larger denominators are greater in size",
        "QUESTION": "What sign would make this statement true?\n\\(\n\\frac{4}{6} ? \\frac{10}{15}\n\\)",
        "CORRECT_ANSWER": "\\( = \\)",
        "STUDENT_WRONG_ANSWER": "\\( < \\)",
        "reasoning": "MISSING_VALUE",
        "PLANING": "MISSING_VALUE"
      },
      {
        "MISCONCEPTION_NAME": "Does not recognise that choosing 2 items at once means the first item does not get replaced in a probability question",
        "QUESTION": "There are \\( 4 \\) red balls and \\( 3 \\) yellow balls in a cup ![A cup with 4 red and 3 yellow balls in it. All of the balls are the same size.]() Kate picks out three balls at the same time.\n\nShe wants to work out the probability all the balls are red.\n\nWhat is the denominator of the final unsimplified answer?",
        "CORRECT_ANSWER": "\\( 210 \